<a href="https://colab.research.google.com/github/dpcks/playdata/blob/main/Machine_Learning/10_%EB%B9%84%EC%A7%80%EB%8F%84%ED%95%99%EC%8A%B5_%EA%B5%B0%EC%A7%91.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 군집화 (clustering)
- 주어진 데이터 집합을 유사한 데이터들의 그룹으로 나누는 것


In [ ]:
import numpy as np
import pandas as pd

In [ ]:
SEED = 42

- 손글씨 데이터받기

In [ ]:
from sklearn.datasets import load_digits
digits = load_digits()
x = digits.data / 255
y = digits.target
x.shape , y.shape

((1797, 64), (1797,))

- 데이터 섞어주기
    - 데이터의 일정한 규칙들이 있을수도 있어서 데이터가 편향적으로 나올수있어 없앨려고 

In [ ]:
np.random.seed(SEED)
idx = np.arange(x.shape[0])
np.random.shuffle(idx)
x = x[idx]
y = y[idx]

K-means
- K개의 랜덤한 중심점으로부터 가까운 데이터들을 묶는 군집화 기법
- 평균을 사용함에 따라 이상치에 민감하다
- 거리를 재기 때문에 스케일링 필수 
- 초기 중심에 민감
- K-means++
    - 초기 중심점 선정의 어려움을 해결하기 위한 방법
- 주요 파라미터
    - n_clusters : 군집수
    - init : 기본값이 k-means++
    

In [ ]:
from sklearn.cluster import KMeans 
kmeans = KMeans(n_clusters=10,random_state=SEED)
kmeans.fit(x) # 비지도학습들은 x값만 넣어줘도 된다. y값 안넣어줘도 됨

KMeans(n_clusters=10, random_state=42)

In [ ]:
kmeans.labels_ # 클래스 정답번호와 군집번호는 전혀 다르다 / 같은 클래스를 같은 군집번호를 넣으면 그건 잘했다고 볼수 있다

array([4, 9, 1, ..., 3, 3, 2], dtype=int32)

In [ ]:
kmeans.predict(x) # 학습데이터를 넣었기 때문에 위에와 같은 값이 나온다

array([4, 9, 1, ..., 3, 3, 2], dtype=int32)

# 대표적인 군집화 성능 평가지표


# 사전에 정의된 그룹이 있는 경우
- 예시 )정답 레이블이 있다 , 그룹을 나타내는 컬럼이 있다.



- homogeneity
    - 각 군집(예측값)이 동일한 클래스(실제값)로 구성되어있는 정도
    - 1에 가까울수록 군집화가 잘되있다

In [ ]:
from sklearn.metrics import homogeneity_score
homogeneity_score(y,kmeans.labels_)

0.7378058968326526

- completeness
    - 각 클래스(실제값)에 대하여 동일한 군집(예측값)으로 구성되어 있는 정도
    - 1에 가까울수록 군집화가 잘되있다

In [ ]:
from sklearn.metrics import completeness_score
completeness_score(y,kmeans.labels_)

0.7458562193736206

- v-measure 
    - homogeneity와 completness 조화평균
    - 1에 가까울수록 군집화가 잘되었다는 의미
    

In [ ]:
from sklearn.metrics import v_measure_score
v_measure_score(y,kmeans.labels_)

0.7418092176543222

## 사전에 정의된 그룹이 없는 경우
- silhouette coefficent(실루엣 계수)
    - 군집내의 거리는 짧을수록, 군집간의 거리는 길수록 좋은 군집화이다. 이 부분을 평가하기 위한 지표
    - -1 ~ 1 의 값을 가진다.
        - 1에 가까울 수록 명확하게 군집화됨을 의미
        - 0에 가까울 수록 군집간의 거리가 가까워 진다는 것을 의미
        - 음수일 경우 잘못된 방식으로 군집화 되었다는 의미


In [ ]:
from sklearn.metrics import silhouette_score
silhouette_score(x,kmeans.labels_)

0.18226446312670963

In [ ]:
def print_score(x,y,pred):
    print(f"homogeneity: {homogeneity_score(y,pred)}")
    print(f"completeness: {completeness_score(y,pred)}")
    print(f"v_measure: {v_measure_score(y,pred)}")
    print(f"silhouette: {silhouette_score(x,pred)}")

In [ ]:
print_score(x,y,kmeans.labels_)

homogeneity: 0.7378058968326526
completeness: 0.7458562193736206
v_measure: 0.7418092176543222
silhouette: 0.18226446312670963


- MiniBatchKMeans
    - mini-batch 개수만큼 무작위로 분리하여 K-means 군집화를 한다.
    - 속도가 훨씬 빠르다
    

In [ ]:
from sklearn.cluster import MiniBatchKMeans 
kmeans = MiniBatchKMeans(n_clusters=10,random_state=SEED,batch_size=512)
kmeans.fit(x)

MiniBatchKMeans(batch_size=512, n_clusters=10, random_state=42)

In [ ]:
print_score(x,y,kmeans.labels_)

homogeneity: 0.7129288903522628
completeness: 0.7288991277425318
v_measure: 0.7208255628252749
silhouette: 0.18472538252636414


# Mean Shift
- K-means 와 유사하게 중심을 이동하면서 군집화하지만 다른점은 평균지점이 아니라, 데이터의 밀도가 가장 높은 지점을 중심으로 이동한다
- 군집개수를 지정해줄수 없다
- 중심점을 기준으로 일정 반경 내를 같은 군집으로 묶는다.
- 반경을 벗어나는 데이터 포인트(샘플, 하나의 행)들에 대해서는 묶지 않는다.
- 이상치를 걸러낼때 사용
- 주요 파라미터
    - bandwidth : None or Float
        - None 일 경우에는 자동으로 반경을 추정
    - cluster_all : 모든 샘플 군집 부여 여부
        - True일 경우 모든 샘플에 강제로 군집부여
        - False일 경우 반경 밖에 샘플들은 -1 부여
        

In [ ]:
from sklearn.cluster import MeanShift
ms = MeanShift(bandwidth=0.1,cluster_all=False,n_jobs=-1)
ms.fit(x)

MeanShift(bandwidth=0.1, cluster_all=False, n_jobs=-1)

In [ ]:
np.unique(ms.labels_)

array([-1,  0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15,
       16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32,
       33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44])

# DBSCAN
- 밀도기반 클러스터링
- 데이터 샘플들이 몰려있는 지점을 찾아 묶어서 군집화
- 주요 파라미터
    - eps : 이웃으로 판단하는 최대거리
    - min_samples : 이웃 반경내 최소 샘플수
    

- 사진

In [ ]:
from sklearn.cluster import DBSCAN
dbscan = DBSCAN(min_samples=3,eps=0.05,n_jobs=-1)
dbscan.fit(x)

DBSCAN(eps=0.05, min_samples=3, n_jobs=-1)

In [ ]:
np.unique(dbscan.labels_) # 안묶이는 애들 이상치로 -1로 나옴

array([-1,  0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15,
       16, 17, 18, 19, 20, 21, 22, 23, 24])

# OPTICS
- DBSCAN과 유사
    - DBSCAN 을 빠르게 만든것
- 주요 파라미터
    - max_eps : 이웃으로 판단하는 최대거리
    - min_samples : 이웃 반경내 최소 샘플수
    

In [ ]:
from sklearn.cluster import OPTICS
optics = OPTICS(max_eps=0.05, min_samples=3, n_jobs=-1)
optics.fit(x)

OPTICS(max_eps=0.05, min_samples=3, n_jobs=-1)

In [ ]:
np.unique(optics.labels_)

array([-1,  0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15,
       16, 17, 18, 19])

# AgglomerativeClustering


In [ ]:
from sklearn.cluster import AgglomerativeClustering
ac = AgglomerativeClustering(n_clusters=10)
ac.fit(x)

AgglomerativeClustering(n_clusters=10)

In [ ]:
print_score(x,y,ac.labels_)

homogeneity: 0.8575128719504723
completeness: 0.8790955851724198
v_measure: 0.8681701126909083
silhouette: 0.17849659940596496
